In [12]:
import numpy as np
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from preprocess import *
import nltk


In [13]:
def discard_from_set(s, discard_items):
    for item in discard_items:
        s.discard(item)
        
discard_items = ['', ' ']


def parse_tweets(fn):

    f = open(fn, 'r')
    tweets_text = f.read().lower()
    f.close()

    tweets = tweets_text.split('\n!@#$%\n')
    tweets = [tweet.lower().strip() for tweet in tweets]
    tweets = set(tweets)
    
    discard_from_set(tweets, discard_items)

    return tweets

In [14]:
pro_tags = ['HillaryClinton']#, 'prolife', 'defundpp', 'stand4life', 'ChooseLife']
con_tags = ['realDonaldTrump']#, 'StandWithPP', 'reprorights', 'protectchoice']

#pro_tags = ['realDonaldTrump']#, 'prolife', 'defundpp', 'stand4life', 'ChooseLife']
#con_tags = ['HillaryClinton']#, 'StandWithPP', 'reprorights', 'protectchoice']

pro_tags_filter = {tag.lower() for tag in ['#prochoice']}
con_tags_filter = {tag.lower() for tag in ['#prolife']}

pro_tweets = set()
for tag in pro_tags:
    current_tweets = parse_tweets('/home/shreya/workspace/AdvanceProject/Tweets_HT/TweetsProcess/'+tag + '.txt')
    
    pre_processed_tweets = []
    c = 0
    for tweet in current_tweets:
        #if not contains_both_tags(tweet, pro_tags_filter, con_tags_filter):
        tweet = preprocess(tweet).strip()
        tweet = filter_tweets(tweet, tag.lower())
        if len(tweet) > 0: pre_processed_tweets.append(tweet)
        #else:
        #    c += 1

    print 'Number of filtered tweets: {0} in tag: {1}'.format(c, tag)
    print "Total Tweets in pro ", len(pre_processed_tweets)
        
    
    pro_tweets.update(pre_processed_tweets)

Number of filtered tweets: 0 in tag: HillaryClinton
Total Tweets in pro  6206


In [15]:
con_tweets = set()
for tag in con_tags:
    current_tweets = parse_tweets('/home/shreya/workspace/AdvanceProject/Tweets_HT/TweetsProcess/'+tag + '.txt')
    
    pre_processed_tweets = []
    c = 0
    
    for tweet in current_tweets:
        #if not contains_both_tags(tweet, pro_tags_filter, con_tags_filter):
        tweet = preprocess(tweet).strip()
        tweet = filter_tweets(tweet, tag.lower())
        
        if len(tweet) > 0: pre_processed_tweets.append(tweet)
        #else:
        #    c += 1 

    print 'Number of filtered tweets: {0} in tag: {1}'.format(c, tag)
    print "Total Tweets in pro ", len(pre_processed_tweets)
    
    con_tweets.update(pre_processed_tweets)

Number of filtered tweets: 0 in tag: realDonaldTrump
Total Tweets in pro  21365


In [16]:
discard_from_set(pro_tweets, discard_items)
discard_from_set(con_tweets, discard_items)

pro_tweets = list(pro_tweets)
con_tweets = list(con_tweets)

con_start_idx = len(pro_tweets)

tweets = pro_tweets + con_tweets

print 'Number of pro and con tweets: {0}, {1}'.format(len(pro_tweets), len(con_tweets))

Number of pro and con tweets: 6175, 21275


In [17]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    nouns = [word for (word, pos) in nltk.pos_tag(tokens) if pos.startswith('N')] 
    
    return tokens

In [18]:
vectorizer = TfidfVectorizer()
tfidf_vecs = vectorizer.fit_transform(tweets)



/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2645: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [19]:
dir_path = '/home/shreya/workspace/AdvanceProject/Tweets_HT/TweetsProcess/'
#file_out = open(dir_path+'Hillary_thresh_2.txt', 'w')
#file_out1 = open(dir_path+'Trump_thresh_2.txt', 'w')

file_out = open(dir_path+'Hillary_thresh_stop.txt', 'w')
file_out1 = open(dir_path+'Trump_thresh_stop.txt', 'w')

In [20]:
k = 5

for i in range(0, len(pro_tweets)):
#for i in random.sample(range(con_start_idx), 5):
    cosine_similarities = linear_kernel(tfidf_vecs[i:i+1], tfidf_vecs)
    cosine_similarities = cosine_similarities.flatten()
    sorted_idx = np.argsort(cosine_similarities)[::-1]
    
   
        
    topk_pro = []
    topk_con = []

    for idx in sorted_idx:
        if idx < con_start_idx and idx != i:
            if cosine_similarities[idx] >= 0.2:
                tweet_score = str(tweets[idx])#+"\t"+str(cosine_similarities[idx])
                topk_pro.append(tweet_score)
                if len(topk_pro) == k:
                    break
            else:
                break

    for idx in sorted_idx:
        if idx >= con_start_idx and idx != i:
            if cosine_similarities[idx] >= 0.2:
                tweet_score = str(tweets[idx])#+"\t"+str(cosine_similarities[idx])
                topk_con.append(tweet_score)
                if len(topk_con) == k:
                    break
            else:
                break
        
    for tweet in topk_con:
        file_out.write(str(tweets[i])+"\n")
        file_out1.write(str(tweet)+"\n")

In [21]:
file_out.close()
file_out1.close()